<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Workshop: Microsoft SQL Server Machine Learning Services

#### <i>A Microsoft Course from the SQL Server team</i>

## 05 - Phase 3: Modeling

<p style="border-bottom: 1px solid lightgrey;"></p>



<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true"></p>

<br>
<br>
<br>

You're learning to use the Team Data Science Process to create a complete solution, using SQL Server as the platform. The phases in the Team Data Science process are:

<dl>
  <dt>1 - Business Understanding</dt>
  <dt>2 - Data Acquisition and Understanding</dt>
  <dt>3 - Modeling <i>(This module)</i></dt>
  <dt>4 - Deployment</dt>
  <dt>5 - Customer Acceptance and Model Retraining</dt>
<dl>

<p style="border-bottom: 1px solid lightgrey;"></p>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>3.0 Modeling</b>
<br>

In this phase, you'll perform feature engineering, create the experiment runs, and run experiments with various settings and parameters. After selecting the best performing run, you'll create a trained model and save it for operationalization in the next phase.

### Goals for Modeling

- Determine the optimal data features for the machine-learning model.
- Create an informative machine-learning model that predicts the target most accurately.
- Create a machine-learning model that's suitable for production.

### How to do it

- Feature engineering: Create data features from the raw data to facilitate model training.
- Model training: Find the model that answers the question most accurately by comparing their success metrics.
- Determine if your model is suitable for production.


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>3.1 Data Engineering</b></p>

Normally your data needs to be prepared and cleaned before you can start training the Model. In this course, most of the preparations have already been done within the database, but you will still convert certain values to factors, making them categorical, creating rolling windows for the maintenance and other tasks.

You'll use a Notebook environment to work with your model. You're doing that in this course as a learning exercise, but this is also a common way of working with a Data Science project. It's also common to use Notebooks to work with Azure Machine Learning Services, Azure Databricks, Azure Cosmos DB or other tools to explore your data, create an experiment, and create and train a model.

Note: The key to a Hybrid Machine Learning Scenario is that you create that model in one location, store the results as a serialized model (using Python's "Pickle" function, R-serialization, the ONNX standard or other binary streaming) and then bring that model in to a local environment using an engine like SQL Server ML Services or [ML.NET with ONNX](https://blogs.msdn.microsoft.com/dotnet/2018/10/08/announcing-ml-net-0-6-machine-learning-net/) to do the predictions (scoring).



<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Feature Engineering using T-SQL</b></p>

You've selected a K-Means Clustering algorithm for this experiment. [You can learn more about that algorithm here](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/k-means-clustering#understanding-k-means-clustering), and find [a more detailed discussion here](https://www.kdnuggets.com/2019/05/guide-k-means-clustering-algorithm.html). 

The specific features we need for the algorithm separates customers along the following dimensions:

- Return frequency
- Return order ratio (total number of orders partially or fully returned versus the total number of orders)
- Return item ratio (total number of items returned versus the number of items purchased)
- Return amount ration (total monetary amount of items returned versus the amount purchased)

Your options are to get the data into this format, or simply use Transact-SQL statements to present the data to R in the datatypes and aggregations. It's a fairly complex set of data that you need to present to the K-Means algorithm, so run the statements below and see if you can figure out the transformations:

In [0]:
USE Analysis;
GO 

SELECT
  ss_customer_sk AS customer,
  round(CASE WHEN ((orders_count = 0) OR (returns_count IS NULL) OR (orders_count IS NULL) OR ((returns_count / orders_count) IS NULL) ) THEN 0.0 ELSE (cast(returns_count as nchar(10)) / orders_count) END, 7) AS orderRatio,
  round(CASE WHEN ((orders_items = 0) OR(returns_items IS NULL) OR (orders_items IS NULL) OR ((returns_items / orders_items) IS NULL) ) THEN 0.0 ELSE (cast(returns_items as nchar(10)) / orders_items) END, 7) AS itemsRatio,
  round(CASE WHEN ((orders_money = 0) OR (returns_money IS NULL) OR (orders_money IS NULL) OR ((returns_money / orders_money) IS NULL) ) THEN 0.0 ELSE (cast(returns_money as nchar(10)) / orders_money) END, 7) AS monetaryRatio,
  round(CASE WHEN ( returns_count IS NULL                                                                        ) THEN 0.0 ELSE  returns_count                 END, 0) AS frequency
  --cast(round(cast(CASE WHEN (returns_count IS NULL) THEN 0.0 ELSE  returns_count END as double)) as integer) AS frequency
FROM
  (
    SELECT
      ss_customer_sk,
      -- return order ratio
      COUNT(distinct(ss_ticket_number)) AS orders_count,
      -- return ss_item_sk ratio
      COUNT(ss_item_sk) AS orders_items,
      -- return monetary amount ratio
      SUM( ss_net_paid ) AS orders_money
    FROM store_sales s
    GROUP BY ss_customer_sk
  ) orders
  LEFT OUTER JOIN
  (
    SELECT
      sr_customer_sk,
      -- return order ratio
      count(distinct(sr_ticket_number)) as returns_count,
      -- return ss_item_sk ratio
      COUNT(sr_item_sk) as returns_items,
      -- return monetary amount ratio
      SUM( sr_return_amt ) AS returns_money
    FROM store_returns
    GROUP BY sr_customer_sk
  ) returned ON ss_customer_sk=sr_customer_sk;
  GO

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>3.2 Experimenting to Create a Model</b></p>

In order to predict using a Model, you have to first find a function that best describes the dependency between the variables in your dataset. This step is called *training the model*. The training dataset will be a subset of the entire dataset. In the next exercise you are going to create the model using the Features from the previous exercise. 

In production, you will use multiple algorithms and variables (called *hyperparameters*) to find the best model. This is called an *experiment*. Once you are satisfied with that experiment's outputs, you can save it as a trained Model. In this case, you'll focus on one algorithm, and use that. You will, however, see how to evaluate the model, which you would do with each run (experiment).

You'll use one set of data to train your model, and another other to test how well it performed. In this activity you will segment the testing and training data. This is a process you should repeat with different ways of separating this data, since one parameter or another might have factors that unduly weight the results.

Using the K-Means model, you'll group the data. K-Means accepts two inputs: The data itself, and a predefined number *k*, the number of clusters. The output is `k` clusters with input data partitioned among them.

The goal of K-means is to group the items into *k* clusters such that all items in same cluster are as similar to each other as possible, and items not in same cluster are as different as possible. It uses the distance measures to calculate similarity and dissimilarity.

This is how the algorithm works: 

- It randomly chooses *k* points and make them the initial centroids (each cluster has a centroid which basically is the “center” of the cluster). 
- For each point, it finds the nearest centroid and assigns the point to the cluster associated with the nearest centroid
- It then updates the centroid of each cluster based on members in that cluster. Typically, a new centroid will be the average of all members in the cluster
- Finally it repeats steps 2 and 3, until the clusters are stable

The number of clusters has to be predefined and the quality of the clusters is heavily dependent on the correctness of the k value specified. You could just randomly pick a number of clusters, run K-Means and iterate your way to a good number. Or we can use R to evaluate which number of clusters is best for our dataset. Let’s determine the number of clusters using an R client this time, with the database serving as the data source. (You're not meant to run all of this code, but you certainly could, if you make the connection to the database properly). 

You should also check out `sqlmlutils`  - a package designed to [help you interact with SQL databases and execute R or Python code in SQL from an R/Python client](https://github.com/microsoft/sqlmlutils).

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Use R to run K-Means</b></p>

- Review the code below. If you are taking this course in-class, the instructor will explain each of these sections, otherwise, you can look up any parts you don't understand:


In [0]:
/*
# Connection string to connect to SQL Server. Don't forget to replace MyServer with the name of your SQL Server instance:
connStr <- paste("Driver=SQL Server;Server=", " MyServer", ";Database=" , "Analysis" , ";Trusted_Connection=true;" , sep="" );

# The query that we are using to select data from SQL Server, from the last phase:
input_query <- "
	SELECT
  ss_customer_sk AS customer,
  round(CASE WHEN ((orders_count = 0) OR (returns_count IS NULL) OR (orders_count IS NULL) OR ((returns_count / orders_count) IS NULL) ) THEN 0.0 ELSE (cast(returns_count as nchar(10)) / orders_count) END, 7) AS orderRatio,
  round(CASE WHEN ((orders_items = 0) OR(returns_items IS NULL) OR (orders_items IS NULL) OR ((returns_items / orders_items) IS NULL) ) THEN 0.0 ELSE (cast(returns_items as nchar(10)) / orders_items) END, 7) AS itemsRatio,
  round(CASE WHEN ((orders_money = 0) OR (returns_money IS NULL) OR (orders_money IS NULL) OR ((returns_money / orders_money) IS NULL) ) THEN 0.0 ELSE (cast(returns_money as nchar(10)) / orders_money) END, 7) AS monetaryRatio,
  round(CASE WHEN ( returns_count IS NULL                                                                        ) THEN 0.0 ELSE  returns_count                 END, 0) AS frequency
  --cast(round(cast(CASE WHEN (returns_count IS NULL) THEN 0.0 ELSE  returns_count END as double)) as integer) AS frequency
FROM
  (
    SELECT
      ss_customer_sk,
      -- return order ratio
      COUNT(distinct(ss_ticket_number)) AS orders_count,
      -- return ss_item_sk ratio
      COUNT(ss_item_sk) AS orders_items,
      -- return monetary amount ratio
      SUM( ss_net_paid ) AS orders_money
    FROM store_sales s
    GROUP BY ss_customer_sk
  ) orders
  LEFT OUTER JOIN
  (
    SELECT
      sr_customer_sk,
      -- return order ratio
      count(distinct(sr_ticket_number)) as returns_count,
      -- return ss_item_sk ratio
      COUNT(sr_item_sk) as returns_items,
      -- return monetary amount ratio
      SUM( sr_return_amt ) AS returns_money
    FROM store_returns
    GROUP BY sr_customer_sk
  ) returned ON ss_customer_sk=sr_customer_sk
"

# Querying SQL Server using the query above and and getting the results back to data frame customer_returns

# This is also where we define the type for columns we want to select (using colClasses), to make sure that the types are correctly transferred to R
customer_returns <- RxSqlServerData(sqlQuery=input_query,colClasses=c(customer ="numeric" , orderRatio="numeric" , itemsRatio="numeric" , monetaryRatio="numeric" , frequency="numeric" ),connectionString=connStr);

# Transform the data from an input dataset to an output dataset
customer_data <- rxDataStep(customer_returns);

# Look at the data we just loaded from SQL Server
head(customer_data, n = 5);

--- Results
customer orderRatio itemsRatio monetaryRatio frequency
1 75675      0          0      0.071633         4
2 86750      0          0      0.000000         0
3 75343      0          0      0.000000         0
4 49280      0          0      0.000000         0
5 89406      0          0      0.000000         0
6 72426      0          0      0.000000         0
--- End Results

# Determine number of clusters
# Using a plot of the within groups sum of squares, by number of clusters extracted, can help determine the appropriate number of clusters
# We are looking for a bend in the plot. It is at this "elbow" in the plot that we have the appropriate number of clusters
wss <- (nrow(customer_data) - 1) * sum(apply(customer_data, 2, var))
       for (i in 2:20)
       wss[i] <- sum(kmeans(customer_data, centers = i)$withinss)
plot(1:20, wss, type = "b", xlab = "Number of Clusters", ylab = "Within groups sum of squares")

*/


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>3.3 Evaluating the Output</b></p>

Based on the graph this code produces, it looks like *k = 4* would be a good value to try. That will group your customers into 4 clusters. 

Now you have derived the number of clusters to use when clustering:

<br>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://sqlchoice.blob.core.windows.net/sqlchoice/static/images/Elbow_graph.JPG">


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Experiment with Clustering</b></p>

Still in the "Client R" mode, it is time to use K-Means. In this sample, you will be using the function `rxKmeans` which is the K-Means function in the `RevoScaleR` package. You'll follow that with a plot output of the clustering to visualize the result:


In [0]:
/* 
# Output table to hold the customer group mappings. This is a table where the cluster mappings will be saved in the database.

# This table is generated from R
return_cluster = RxSqlServerData(table = "return_cluster", connectionString = connStr);

# Set.seed for random number generator for predictability
set.seed(10);

# Generate clusters using rxKmeans and output key / cluster to a table in SQL Server called return_cluster
clust <- rxKmeans( ~ orderRatio + itemsRatio + monetaryRatio + frequency, customer_returns, numClusters=4
         , outFile=return_cluster, outColName="cluster" , extraVarsToWrite=c("customer"), overwrite=TRUE);

# Read the customer returns cluster table from the DB
customer_cluster <- rxDataStep(return_cluster);

# Plot the clusters (you need to install R library "cluster". If you don't have that installed, just uncomment the next line of code)
# If needed, install the library with: install.packages("cluster")
library("cluster");
clusplot(customer_data, customer_cluster$cluster, color=TRUE, shade=TRUE, labels=4, lines=0, plotchar = TRUE);
*/

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true"><b>3.4 Plotting Customer Data</b></p>

The output plot does not really tell you anything about how the different clusters are different from each other. You had 4 different variables and you need to know something about the different characteristics of each cluster to make conclusions.

Using K-Means is also a data mining method and you still need to spend time on analyzing the results. Depending on your data, you might need to use different methods to analyze the result. Here you just want to show that plotting can be one way. Let’s try something else and see if you can get some more insight from that.

<br>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://sqlchoice.blob.core.windows.net/sqlchoice/static/images/Cluster_plot.JPG">


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Analyze Cluster Means</b></p>

This plot does not really tell us anything about how the different clusters are different from each other. We had 4 different variables and we need to know something about the different characteristics of each cluster to make conclusions.
Using K-Means is just a data mining method and you still need to spend time on analyzing the results. Depending on your data, you might need to use different methods to analyze the result. Here we just want to show that plotting can be one way. Let’s try something else and see if we can get some more insight from that.

The code below approaches the analysis in text, now that you've tried a graph:


In [0]:
/* 
# Look at the clustering details and analyze results
clust

------- Results
Call:
rxKmeans(formula = ~orderRatio + itemsRatio + monetaryRatio +
    frequency, data = customer_returns, outFile = return_cluster,
    outColName = "cluster", extraVarsToWrite = c("customer"),
    overwrite = TRUE, numClusters = 4)
Data: customer_returns
Number of valid observations: 37336
Number of missing observations: 0

Clustering algorithm:
K-means clustering with 4 clusters of sizes 31675, 671, 2851, 2139
Cluster means:
   orderRatio   itemsRatio monetaryRatio frequency
1 0.000000000 0.0000000000    0.00000000  0.000000
2 0.007451565 0.0000000000    0.04449653  4.271237
3 1.008067345 0.2707821817    0.49515232  1.031568
4 0.000000000 0.0004675082    0.10858272  1.186068
Within cluster sum of squares by cluster:
         1          2          3          4
    0.0000  1329.0160 18561.3157   363.2188

------- End Results

*/

The `clust` object contains the results from our K-Means clustering. Focusing on the cluster mean values, it seems like you can actually interpret something. Just to refresh your memory, here are the definitions of your variables:

- frequency = return frequency
- orderRatio = return order ratio (total number of orders partially or fully returned versus the total number of orders)
- itemsRatio = return item ratio (total number of items returned versus the number of items purchased)
- monetaryRatio = return amount ratio (total monetary amount of items returned versus the amount purchased)

So what do the mean values tell you? Well, cluster `1` (the largest cluster) seems to be a group of customers that are not active. All values are zero. And cluster `3` seems to be a group that stands out in terms of return behavior.

Data mining using K-Means often requires further analysis of the results, and further steps to better understand each cluster, but it provides some very good leads. Cluster `1` is a set of customers who are clearly not active. Perhaps your company can target marketing efforts towards this group to trigger an interest for purchases.

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

 - Learn more about Feature Engineering and Modeling here: https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/lifecycle-modeling


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Phase 4 - Deployment</p>

After your experimentation phase, you'll continue working through the Team Data Science Process in the next phase - *06 - Phase 4 - Deployment*. Open that Notebook to continue.